# Assessing Player Value through Quantitative Measures of Match Performance

## CS 109 Final Project by FIFA (Future Impact Football Analytics)
##### Presented by Taylor Killian, Virgile Audi and Abhishek Malali

#Table of Contents
* [Assessing Player Value through Quantitative Measures of Match Performance](#Asessing-Player-Value-through-Quantitative-Measures-of-Match-Performance)
	* [Section 0: Project Overview](#Section 0:-Project-Overview)
		* [Background and Motivation](#Background-and-Motivation)
		* [Project Objectives](#Project-Objectives)
	* [Section 1: Gathering Data: Acquisition and Cleaning](#Section 1:-Gathering-Data:-Acquisition-and-Cleaning)
		* [1.1: Scraping Data](#1.1:-Scraping-Data)
			* [Match Statistics](#Match-Statistics)
			* [Transfer Values](#Transfer Values)
		* [1.2: Data Cleaning](#1.2:-Data-Cleaning)
            * [Calculation of Player Impact](#Calculation-of-Player-Impact)
	* [Section 2: Feature Extraction and ML](#Section-2:-Feature-Extraction-and-ML)
        * [2.1: Continued Data Munging](#2.1:-Continued-Data-Munging)
		* [2.2: EDA via Methods of ML](#2.2: EDA-via-Methods-of-ML)
            * [Validating our Metric](#Validating-out-Metric)
            * [Clustering](#Clustering)
	* [Section 3: Further Analysis and Visualization](#Section 3:Further-Analysis-and-Visualization)
    * [Section 4: Conclusions](#Section 4:-Conclusions)


In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

## Section 0: Project Overview

### Background and Motivation

The three of us are large football (and by football, we of course mean the one and only, the one played with feet) fans. We consume the game in many aspects of our lives. Our hearts are lifted when our respective teams do well and are similarly frustrated when they fail to perform to our expectations. When teams consistently underperform they look to infuse their team with more talented players, typically drawn from lower-tier or financially poor teams. The financial incentive for both teams in any transaction is clear, the buying team has hope that the new addition will put them over the top talent-wise to improve their standing in their league competition while the selling team may look at the new funds as a way to prepare for the future and secure several new players to improve their team in the future.

In the past few years the economy of player transfers has undergone significant inflation where the top-tier clubs have shown a willingness to overpay for marginal talent or for players that may not be a good fit in their team. There is some notion that the environment is broken and is in need of revitalization. There are multiple ways to address the psychology of club executives and try to identify their motivations for spending money to procure new talent for their team. We refrain from any investigation into club finances but want to determine a method by which we can accurately and objectively valuate any player based on the impact that they have on the matches they play in.

### Project Objectives

Our primary motivation is to quantify the impact (based on goals scored, assists made and consistency of play) an individual player has on the games which he plays for his team. We intend to use this quantity for each player to infer their value (agnostic of team at first and later based on current team and potential buyers). We hope to learn more about what goes into modeling player value, beyond subjective measures of team they currently play for and the league they currently play in, and how we can compare like players with each other. Our hope is that this work will provide the foundation to build an analysis portfolio to enable application to sports analytics positions or a long-term hobby at the very least.

## Section 1: Gathering Data: Acquisition and Cleaning

### 1.1: Scraping Data

#### Match Statistics

Link to [Notebook 2](Scraping/scraping.ipynb)

#### Transfer Values

### 1.2 Data Cleaning

#### Calculation of Player Impact

## Section 2: Feature Extraction and ML

### 2.1: Continued Data Munging

### 2.2: EDA via Methods of ML

#### Validating our Metric

#### Clustering

## Section 3: Further Analysis and Visualization

## Section 4: Conclusions